## Load lib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyramses

## Load Models

In [ ]:
from agc_models import end_simulation
from agc_models import move_file
from agc_models import sfc
from agc_models import Gens
from agc_models import sortGens
from agc_models import multipleIncreaseDelay

from agc_models import chop_curve
from agc_models import rate_of_change
from agc_models import settling_time
from agc_models import curve_of_rate_of_change

## Load a simulator instance

In [ ]:
ram = pyramses.sim()

## Load saved test-case & Add more observation(s)

In [ ]:
case = pyramses.cfg('cmd.txt')
case.addRunObs('MS CCGTG2.01')  # Monitor, will write real-time (voltage-time or) frequency-time data of bus G2 (GBdynHVDC.dat)
monitor = ['CCGTG2.01']

## Universe Parameters. Start: 300sec, Settling before: 1200sec (+900sec = +15min), End: 1500sec; Time step: 1sec.

In [ ]:
prepared_folder_address = 'cur'
nominal_frequency = 1.0
start_time = 300.0
settling_end_time = 1200.0
end_time = 1500.0  # (dst file)
agcTimeStep = 1.0

## Breaker (one of distrubances) & Selected generators (to help fix errors)

In [ ]:
breaker = 'CCGTG21.01' # (dst file)

# gen's code     name       weight  mini delay              
gen15_2 = Gens('COALG15.02', 0.161, 0.0615)  # 8027.401/27580.102 = 0.291
gen16_1 = Gens('CCGTG16.01', 0.226, 0.1220)  # 7631.982/27580.102 = 0.277
gen17_2 = Gens('COALG17.02', 0.161, 0.1221)  # 2240.000/27580.102 = 0.081
gen23_1 = Gens('CCGTG23.01', 0.226, 0.0609)  # 5716.446/27580.102 = 0.207
gen26_1 = Gens('CCGTG26.01', 0.226, 0.0602)  # 3964.273/27580.102 = 0.143

liST = []
liST.append(gen15_2)
liST.append(gen16_1)
liST.append(gen17_2)
liST.append(gen23_1)
liST.append(gen26_1)
list_of_gens, weight_of_gens,  mini_list_of_td = sortGens(liST)
# print(list_of_gens)
# print(weight_of_gens)
print(mini_list_of_td)

## Test with fixed kp and ki

In [ ]:
if __name__ == '__main__':
    kp = 0.6
    ki = 0.03
    list_of_td = multipleIncreaseDelay(100, mini_list_of_td)  # 100%
    
    kp = "{0:.4f}".format(round(float(kp),4))
    ki = "{0:.4f}".format(round(float(ki),4))
    print("kp = " + str(kp))
    print("ki = " + str(ki))
    print(list_of_td)

    sfc(ram, case, monitor, list_of_gens, weight_of_gens, list_of_td, prepared_folder_address, start_time, end_time, agcTimeStep, breaker, nominal_frequency, kp, ki)

## Embed the frequency/voltage response in the output
**https://pyramses.paristidou.info/_modules/pyramses/extractor.html#extractor.getSync**

In [ ]:
ext = pyramses.extractor(case.getTrj())

## A. Check frequency in the grid via monitor

In [ ]:
t_ori, f_ori, t, f = chop_curve(ext.getSync('CCGTG2.01').S.time, ext.getSync('CCGTG2.01').S.value, chop_value=300)
realSettlingTime = settling_time(t, f, 1/5000, nominal_frequency)
# ext.getSync('CCGTG2.01').S.plot()
fig = plt.figure(figsize=(20,10))
plt.plot(t, f, label = ext.getSync('CCGTG2.01').S.msg + ": settling time = " + str(realSettlingTime) + " sec.")
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('frequency (pu)')
plt.grid(True)

## B. Check power & power rate of the five generators (power rate = delta power / delta time, should be around 10% (CCGT) or 5% (COAL) of power capacity per min)

### 1. Pnom(G15.02) = 6823.291 MW

In [ ]:
# ext.getSync('COALG15.02').P.plot()
fig = plt.figure(figsize=(20,10))
plt.plot(ext.getSync('COALG15.02').P.time, ext.getSync('COALG15.02').P.value, label = ext.getSync('COALG15.02').P.msg)
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.grid(True)

### COALG15.02: 5% capacity/min = 341.165 MW/min

In [ ]:
curve_of_rate_of_change('COALG15.02', ext.getSync('COALG15.02').P.time, ext.getSync('COALG15.02').P.value, start_time, settling_end_time, agcTimeStep, unit_converter=60)

### 2. Pnom(G16.01) = 6868.783 MW

In [ ]:
# ext.getSync('CCGTG16.01').P.plot()
fig = plt.figure(figsize=(20,10))
plt.plot(ext.getSync('CCGTG16.01').P.time, ext.getSync('CCGTG16.01').P.value, label = ext.getSync('CCGTG16.01').P.msg)
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.grid(True)

### CCGTG16.01: 10% capacity/min = 686.878 MW/min

In [ ]:
curve_of_rate_of_change('CCGTG16.01', ext.getSync('CCGTG16.01').P.time, ext.getSync('CCGTG16.01').P.value, start_time, settling_end_time, agcTimeStep, unit_converter=60)

### 3. Pnom(G17.02) = 1904.000 MW

In [ ]:
# ext.getSync('COALG17.02').P.plot()
fig = plt.figure(figsize=(20,10))
plt.plot(ext.getSync('COALG17.02').P.time, ext.getSync('COALG17.02').P.value, label = ext.getSync('COALG17.02').P.msg)
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.grid(True)

### COALG17.02: 5% capacity/min = 95.200 MW/min

In [ ]:
curve_of_rate_of_change('COALG17.02', ext.getSync('COALG17.02').P.time, ext.getSync('COALG17.02').P.value, start_time, settling_end_time, agcTimeStep, unit_converter=60)

### 4. Pnom(G23.01) = 4858.979 MW

In [ ]:
# ext.getSync('CCGTG23.01').P.plot()
fig = plt.figure(figsize=(20,10))
plt.plot(ext.getSync('CCGTG23.01').P.time, ext.getSync('CCGTG23.01').P.value, label = ext.getSync('CCGTG23.01').P.msg)
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.grid(True)

### CCGTG23.01: 10% capacity/min = 485.898 MW/min

In [ ]:
curve_of_rate_of_change('CCGTG23.01', ext.getSync('CCGTG23.01').P.time, ext.getSync('CCGTG23.01').P.value, start_time, settling_end_time, agcTimeStep, unit_converter=60)

### 5. Pnom(G26.01) = 3567.846 MW

In [ ]:
# ext.getSync('CCGTG26.01').P.plot()
fig = plt.figure(figsize=(20,10))
plt.plot(ext.getSync('CCGTG26.01').P.time, ext.getSync('CCGTG26.01').P.value, label = ext.getSync('CCGTG26.01').P.msg)
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('power (MW)')
plt.grid(True)

### CCGTG26.01: 10% capacity/min = 356.785 MW/min

In [ ]:
curve_of_rate_of_change('CCGTG26.01', ext.getSync('CCGTG26.01').P.time, ext.getSync('CCGTG26.01').P.value, start_time, settling_end_time, agcTimeStep, unit_converter=60)